# Lab-1 Task3

Move the robot through a set of poses in both Cartesian and joint space using the Python API within RoboDK. More information on this can be found at this link: https://robodk.com/offline-programming

## Task steps

* Open RoboDK
* Open the "ur5_and_table_and_gripper.rdk" file from Task 2.
* Create a new Python program by selecting the button shown in Figure 17. Right click on the new program (most likely called Prog1) and click “Edit Python Script”.
* Copy and paste the Python script provided on the course website into the window that pops up. It is not crucial that you understand what the beginning portion of this script does, but make sure to pay attention to how the poses and movements are defined.
* Save this program as Task_3.py
* Right click on the UR5 robot and select “Connect to robot” and follow the steps outlined in the Appendix to connect to the robot (you already did this in Task 2, but it’s good to double check).
* To simulate the robot’s movements in RoboDK and check that the robot will move where
expected, right click the script “Task_3.py” and select “Run Python script”.
* Once you are satisfied with the robot's movements, right click on “Task_3.py” and select
“Run on robot”. This will run on the actual UR5 as the Python program is executed. If “run on robot” doesn’t seem to work, try clicking “start on robot” instead.


In [ ]:
# This macro shows an example of running a program on the robot using the Python API (online programming)
# More information about the RoboDK API appears here:
# https://robodk.com/doc/en/RoboDK-API.html
from robolink import *    # API to communicate with RoboDK
from robodk import *      # robodk robotics toolbox
# Any interaction with RoboDK must be done through RDK:
RDK = Robolink()
# Select a robot (a popup is displayed if more than one robot is available)
robot = RDK.ItemUserPick('Select a robot', ITEM_TYPE_ROBOT)
if not robot.Valid():
    raise Exception('No robot selected or available')
RUN_ON_ROBOT = False
# Important: by default, the run mode is RUNMODE_SIMULATE
# If the program is generated offline manually the runmode will be RUNMODE_MAKE_ROBOTPROG,
# Therefore, we should not run the program on the robot
if RDK.RunMode() != RUNMODE_SIMULATE:
    RUN_ON_ROBOT = False
if RUN_ON_ROBOT:

    # Connect to the robot using default IP
    success = robot.Connect() # Try to connect once
    status, status_msg = robot.ConnectedState()
    if status != ROBOTCOM_READY:
        # Stop if the connection did not succeed
        print(status_msg)
        raise Exception("Failed to connect: " + status_msg)
    # This will set to run the API programs on the robot and the simulator (online programming)
    RDK.setRunMode(RUNMODE_RUN_ROBOT)

# Get the current joint position of the robot
# (updates the position on the robot simulator)
joints_ref = robot.Joints()
# get the current position of the TCP with respect to the reference frame:
# (4x4 matrix representing position and orientation)
target_ref = robot.Pose()
pos_ref = target_ref.Pos()
# It is important to provide the reference frame and the tool frames when generating programs offline
# It is important to update the TCP on the robot mostly when using the driver
robot.setPoseFrame(robot.PoseFrame())
robot.setPoseTool(robot.PoseTool())
#robot.setZoneData(10) # Set the rounding parameter (Also known as: CNT, APO/C_DIS, ZoneData, Blending radius, cornering, ...)
robot.setSpeed(200) # Set linear speed in mm/s

# Setting Pose 1, Pose 2, and Pose 3 from the Lab Manual
# We enter our xyzrpw (X, Y, Z, RX, RY, RZ) into the function xyzrpw_2_pose()
# to convert out position and euler angles to a 4x4 pose matrix
# Start at the home position
home = [0, -90, 0, -90, 0, 0]
pose_1 = xyzrpw_2_pose([-575, -350, 300, 131.441, -5.933 , 58.398]) 
pose_2 = xyzrpw_2_pose([-240, -445, 650, 107.452, 23.626, -107.452])
pose_3 = xyzrpw_2_pose([400, -400, 200, 160.148, -1.116, -6.369])
# Defined position between joint and linear movements to realign robot
in_between = [0, -60, 30, -90, -90, -90]
# Move the robot in joint space using MoveJ
robot.MoveJ(home)
robot.MoveJ(pose_1)
robot.MoveJ(pose_2)
robot.MoveJ(pose_3)
# Intermediate move used to place robot in correct configuration
# robot.MoveJ(in_between) --> Option 1 to reconfigure robot
robot.MoveJ(home) # Option 2
# Wait 3 seconds to run in cartesian space using MoveL
pause(3)
# Using Option 2, a joint movement is made to pose 1 to begin linear pattern
# Pose 2 is thus the start of the linear patter and pose 1 is hit last
# Alternatively, with Option 1, the robot is configured to move linearly to pose 1 right away
# Option 2 was selected for a cleaner set of movements
robot.MoveJ(pose_1)
robot.MoveL(pose_2)
robot.MoveL(pose_3)
robot.MoveL(pose_1)
robot.MoveJ(home)

print('Done')

# Lab-1 Task5

In this task, the robot is equipped with an aluminum semi-cylinder as its end effector as seen in Figure 19. Another semi-cylinder is attached via four load cells to two t-slot extrusions mounted to the table. The robot will collide with this semi-cylinder, and the force will be recorded. The purpose of this task is for you to examine what happens when the cobot hits an unexpected “wall”. Although you are not conducting the experiment, the steps are outlined below for your reference so that you can understand how the experiment was performed. This will help you with your analysis.

# Task steps

* Prepare the Python code in order to configure the robot as shown in Figure 19. The two aluminum semi-cylinders should be perpendicular to each other. The robot should be positioned around [X= 300, Y= -420, Z=200].
* Have the robot move along a straight line (using MoveL) to a point beyond the collision fixture. This allows the robot to fully collide with the fixture during which the force will be recorded.
* BEFORE running your code on the robot, run your code offline on RoboDK. This allows you to ensure that the robot is doing what you expected. This is good practice and should be done for all the laboratory modules.
* Run the following experiment three times at each speed setting in a range from 0 to 1.2 m/s. To do this, move the end effector so that it collides with the collision fixture at speed increments of 5% of the max speed (1.2 m/s) between 0 and 0.6 m/s and in 10% increments between 0.6 and 1.2 m/s. Be sure to record the data using the PuTTY software. (see the Miscellaneous Instructions and Troubleshooting Guide on the course web site’s Lab Assignments page for how to use PuTTY). 

For reference, the collision speeds that were run are; 0.06, 0.12, 0.18, 0.24, 0.30, 0.36, 0.42, 0.48, 0.54, 0.60, 0.72, 0.84, 0.96, 1.08, 1.29 m/s. 

You will be given the results of this experiment which can be downloaded from the course webpage. You will be analyzing these results as described in the deliverables section.

In [ ]:
%% Dingyi Gu
%ME 5286- Robotics
%Instructor: Dr. Max Donath

%Department of Mechanical Engineering
%gu002913@umn.edu
%+1 (610)-350-8248
%Feb.12
%2020

clc
clear all
close all

%% Task 5 collision data processing

T = readtable('ME5286_CollisionData_2020.csv');

Sp = table2array(T(:,1));
Fc = table2array(T(:,2));

figure(11)
plot(Sp,Fc,'o-');
grid on;
title('CollisionData postprocessing');
xlabel('Speed of the robot(m/s)');
ylabel('Average force exerted by the robot (N)');

# Lab-1 Memo

In [ ]:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% Memo
% LaTeX Template
% Version 1.0 (30/12/13)
%
% This template has been downloaded from:
% http://www.LaTeXTemplates.com
%
% Original author:
% Rob Oakes (http://www.oak-tree.us) with modifications by:
% Vel (vel@latextemplates.com)
%
% License:
% CC BY-NC-SA 3.0 (http://creativecommons.org/licenses/by-nc-sa/3.0/)
%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

\documentclass[letterpaper,11pt]{texMemo} % Set the paper size (letterpaper, a4paper, etc) and font size (10pt, 11pt or 12pt)

\usepackage{parskip} % Adds spacing between paragraphs
\setlength{\parindent}{15pt} % Indent paragraphs

%----------------------------------------------------------------------------------------
%	MEMO INFORMATION
%----------------------------------------------------------------------------------------

\memoto{Dr. Max Donath} % Recipient(s)

\memofrom{Mr. Dingyi Gu} % Sender(s)

\memosubject{ME 5286 Robotics Lab1} % Memo subject

\memodate{Tuesday, February 11, 2020} % Date, set to \today for automatically printing todays date
 
\logo{\includegraphics[width=0.3\textwidth]{logo.eps}} % Institution logo at the top right of the memo, comment out this line for no logo

%----------------------------------------------------------------------------------------



\renewcommand{\thesubsection}{\arabic{subsection}}
\makeatletter
\def\@seccntformat#1{\@ifundefined{#1@cntformat}%
	{\csname the#1\endcsname\quad}%    default
	{\csname #1@cntformat\endcsname}}% enable individual control
\newcommand\section@cntformat{}     % section level 
\makeatother


\usepackage{graphicx} %This one allows you to import images.
\usepackage{float} %Allows for control of float positions


%subfig
\usepackage{caption}

\usepackage{subfig}
\usepackage{setspace}


\begin{document}

\maketitle % Print the memo header information

%----------------------------------------------------------------------------------------
%	MEMO CONTENT
%----------------------------------------------------------------------------------------


\doublespacing

\section{Task1-3}

\subsection{Describe the difference between joint and Cartesian space and how that affected the	motion of the robot.}

Cartesian space is defined by the position and orientation of the end effector of a robot, which is broken down into X, Y and Z coordinates in a simple grid. While joint space is defined by a vector whose components are the translational and angular displacements of each joint of a robotic link. When applying Cartesian space along the kinematic chain, often you will analyze the position of one link with respect to the next link as a new coordinate system. However, this kind of information is not always particularly helpful in robotics. Depending on where it is, there might be a number of ways to position its end effector in a certain location, and it could be hard to tell a robotic arm to reach there. So instead, you tell each of its joints exactly what position to be in. By giving robot the angles each joint should be, the robot can then move to the desired location. 

\subsection{Describe the limitations of the robot’s workspace based upon the mounting location of the robot on the table.}

When you add a target which is quite close to the workspace, the UR5 robot may touch the table because of different configurations (like the arm is elbow down) in both RoboDK simulation and real situation, where the robot should stop its motion. As a result, it is really important for us to check and modify the configuration once the target is added. Otherwise the robot will generate configuration in relation to the embedded algorithm, which contains the collision between the robot and the table. It is essential to double check it before implementing the program linked to the selected robot.

\subsection{Show the usable workspace of the robot. Use a program (Matlab, Excel, etc.) to plot the workspaces. This can either be a 3D view or multiple 2D views.}

%\begin{figure}[H]
%	\centering
%	\includegraphics[height=1.4in]{C:/Document_Dingyi/UMN/Spring_2020/ME5286/Lab1/Lab_memo/figures/Left.png}
%	\caption{System Block Diagram}
%	\label{fig:1}
%\end{figure} 	

%C:/Document_Dingyi/UMN/Spring_2020/ME5286/Lab1/Lab_memo/figures/Left.png

\begin{figure}[H]
	\centering   
	\subfloat[Front view]{
		\label{fig:1:a} %% label for first subfigure
		\includegraphics[height=2in]{C:/Document_Dingyi/UMN/Spring_2020/ME5286/Lab1/Lab_memo/figures/Left.png}}
	\hspace{0.1in} 
	\subfloat[Side view]{
		\label{fig:1:b} %% label for second subfigure
		\includegraphics[height=2in]{C:/Document_Dingyi/UMN/Spring_2020/ME5286/Lab1/Lab_memo/figures/Back.png}}
	\hspace{0.1in}
	\subfloat[Top view]{
		\label{fig:1:c} %% label for second subfigure
		\includegraphics[height=2in]{C:/Document_Dingyi/UMN/Spring_2020/ME5286/Lab1/Lab_memo/figures/Top.png}}
	\caption{Usable workspace of the robot UR5 } 
	\label{fig:1}
\end{figure}

Fig.~\ref{fig:1} presents the edges of the usable workspace, and the robot cannot go under the table and behind the wall as well.


\subsection{What are the two safety planes? And how did you determine where they were?}

The first one is offset from the table surface corresponding to the usable workspace in qn3. The second one is behind the robot and parallel to the wall.



\subsection{Is the robot elbow up or elbow down for these tasks? How do you know and how did you ensure that the robot chose this elbow configuration?}

Initially at some targets, the robot was elbow down. We modified the elbow by clicking configuration at different waypoints in order to make sure there is no collision between the elbow and the table.


\subsection{Define what “collaborative” means when talking about “collaborative robotics”? What makes the robot collaborative?}

Collaborative robot is the robot that is able to work or interact with human in a shared workspace. “Collaborate” means there are some features for cobot: safety monitored stop, hand guiding, speed and separation monitoring, power and force limiting. Therefore, “collaborative” defines that the cobot should be designed by working
together with human. Cobot is able to monitor the space between itself and human and stop moving when it might cause danger to people. For example, when people step into pre-designed safety zone, the robot will slow down the speed or stop when getting too
close. The power and force limiting feature of robot can detect the overload force on itself and slow down or stop when needed. Therefore, people can safely work with cobot without any additional safety devices.

\subsection{Compare and contrast the three programming methods. Which do you find easiest to use at the moment? Which method seems to have the most capabilities or lack of capabilities? Which method do you foresee yourself using for the rest of the class?}

The easiest one to use at the moment is RoboDK. RoboDK Python API has the most capability. PolyScope has lack of capability because every step and motion of robot need to be manually controlled by myself using the tablet. I preferred to work with RoboDK Python API for the rest of the class.


\section{Task4}

\subsection{Create a table showing the vertices you used in the cube and describe how you picked	these vertices}


\renewcommand\arraystretch{2}
\begin{table}[H]\footnotesize
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
 & \textbf{X($mm$)}&\textbf{Y($mm$)}&\textbf{Z($mm$)}& \textbf{RotX($deg$)} & \textbf{RotY($deg$)} & \textbf{RotZ($deg$)} \\
\hline
Home & 0.000 & -191.000 & 1000.280 & 90.000 & 0.000 & 180.000\\
\hline
Target1 & -300.000 & -600.000 & 600.000 & 90.000 & 0.000 & 180.000\\
\hline
Target2 & -200.000 & -600.000 & 600.000 & 90.000 & 0.000 & 180.000\\
\hline
Target3 & -200.000 & -600.000 & 520.000 & 90.000 & 0.000 & 180.000\\
\hline 
Target4 & -300.000 & -600.000 & 520.000 & 90.000 & 0.000 & 180.000\\
\hline
Target5  & -300.000 & -100.000 & 520.000 & 90.000 & 0.000 & 180.000\\
\hline
Target6  & -300.000 & -100.000 & 600.000 & 90.000 & 0.000 & 180.000\\
\hline
Target7  & -200.000 & -100.000 & 600.000 & 90.000 & 0.000 & 180.000\\
\hline
Target8  &-200.000 & -100.000 & 520.000 & 90.000 & 0.000 & 180.000\\
\hline
\end{tabular}
\caption{Vertices picked for maximum length in y-direction}
\label{table:1}
\end{table}

Table.~\ref{table:1} shows the vertices I picked for the maximum length in y-direction. To begin with, the robot started from the home position to Target1. In order to form the complete prism motion in 100$mm$ $\times$ 500$mm$ $\times$ 80$mm$, the robot moved from 1-2-3-4-5-6-7-8-3-2-7-6-1. Fig.~\ref{fig:2:a} shows the trajectory in y-direction.

%\begin{figure}[H]
%	\centering
%	\includegraphics[height=2.4in]{C:/Document_Dingyi/UMN/Spring_2020/ME5286/Lab1/Lab_memo/figures/task4y.png}
%	\caption{Taks4 in y-direction}
%	\label{fig:2}
%\end{figure} 	

\begin{figure}[H]
	\centering   
	\subfloat[Front view]{
		\label{fig:2:a} %% label for first subfigure
		\includegraphics[height=2.2in]{C:/Document_Dingyi/UMN/Spring_2020/ME5286/Lab1/Lab_memo/figures/task4y.png}}
	\hspace{0.1in} 
	\subfloat[Side view]{
		\label{fig:2:b} %% label for second subfigure
		\includegraphics[height=2.2in]{C:/Document_Dingyi/UMN/Spring_2020/ME5286/Lab1/Lab_memo/figures/task4z.png}}
	\caption{Task 4 } 
	\label{fig:2}
\end{figure}


\renewcommand\arraystretch{2}
\begin{table}[H]\footnotesize
	\centering
	\begin{tabular}{|c|c|c|c|c|c|c|}
		\hline
		& \textbf{X($mm$)}&\textbf{Y($mm$)}&\textbf{Z($mm$)}& \textbf{RotX($deg$)} & \textbf{RotY($deg$)} & \textbf{RotZ($deg$)} \\
		\hline
		Target1 & 400.000 & -200.000 & 800.000 & 90.000 & 0.000 & 180.000\\
		\hline
		Target2 & 200.000 & -200.000 & 800.000 & 90.000 & 0.000 & 180.000\\
		\hline
		Target3 & 200.000 & -200.000 & 300.000 & 90.000 & 0.000 & 180.000\\
		\hline 
		Target4 & 400.000 & -200.000 & 300.000 & 90.000 & 0.000 & 180.000\\
		\hline
		Target5  & 400.000 & -400.000 & 300.000 & 90.000 & 0.000 & 180.000\\
		\hline
		Target6  & 200.000 & -400.000 & 300.000 & 90.000 & 0.000 & 180.000\\
		\hline
		Target7  & 200.000 & -400.000 & 800.000 & 90.000 & 0.000 & 180.000\\
		\hline
		Target8  &-400.000 & -400.000 & 800.000 & 90.000 & 0.000 & 180.000\\
		\hline
	\end{tabular}
	\caption{Vertices picked for maximum length in z-direction}
	\label{table:2}
\end{table}


Table.~\ref{table:2} shows the vertices picked by my partner Steven. The trajectory was presented in Fig.~\ref{fig:2:b}



\subsection{What does that tell you about the robot’s workspace?}

When picking the vertices of prism, some targets might be not reachable for the robot arm. We need make sure these targets are not too far from or too close to the home position. In addition, you cannot stretch too much on z or
y direction because the target might go under the table or behind the wall. Run the simulation before starting on the real robot to fix errors. 


\subsection{Include a screenshot or figure of the robot trajectory tracing the two cubes.}

Please see details in Fig.~\ref{fig:2}.

\cleardoublepage

\section{Task5}

\subsection{Create a graph showing the force exerted by the robot vs. speed of the robot. (To do this, average the three forces measured at each speed and plot this average force vs speed)}

\begin{figure}[H]
	\centering
	\includegraphics[height=2.5in]{C:/Document_Dingyi/UMN/Spring_2020/ME5286/Lab1/Lab_memo/figures/task5.png}
	\caption{Taks5}
	\label{fig:3}
\end{figure} 	

Fig.~\ref{fig:3} presents the relationship between average force and speed during the collision experiment.



\subsection{What does this graph say about the UR5 strategy for preventing excessive forces when colliding?}

According to Fig.~\ref{fig:3}, when the speed increases, there is a huge drop for force exerted on the robot from (0,0.2$m/s$) , which means the colliding force decreases significantly in order to prevent from excessive damage. This phenomenon help cancel the effect made by speed which is larger than 0.2$m/s$ and protect the robot itself.


\subsection{What was the maximum force recorded and at which speed?}

The maximum force was recorded as 250.12$N$ at 0.06 $m/s$.




%----------------------------------------------------------------------------------------

\end{document}